### Creating a workspace using notebook

In [ ]:
#import worskpace constructor and create a workspace

# from azureml.core import Workspace
# lqai_ws = Workspace.create(name='workspace-name',
#                             subscription_id='your-subscription-id',
#                             resource_group='your-resource-group',
#                             create_resource_group=False,
#                             location='Southeast Asia')

### Accessing workspace already created

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name='workspace-name',
                subscription_id='your-subscription-id',
                resource_group='your-resource-group',)
ws

In [ ]:
ws.get_details()

### Writing train.py, a script to train model

In [ ]:
model_name = "reg_model.pkl"
model_dir = "outputs"

In [ ]:
%%writefile train.py
# trains a regression model and saves it

import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

#loads the data set
diabetes_dataset = load_diabetes()

x_train, x_test, y_train, y_test = train_test_split(diabetes_dataset.data, diabetes_dataset.target)


# implements linear regression model
reg_model = LinearRegression()
reg_model.fit (x_train, y_train)

# saves the trained model
os.makedirs("outputs", exist_ok=True)
model_name = "reg_model.pkl"
model_dir = "outputs"
joblib.dump(value=reg_model, filename=model_dir + "/" + model_name)

In [ ]:
from azureml.core.compute import AmlCompute

# list of all vms that your workspace supports
list_vms = AmlCompute.supported_vmsizes(workspace=ws)
list_vms

In [ ]:
from azureml.core.runconfig import RunConfiguration

compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "Standard_DS1_v2"

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.20.3")
dependencies.add_pip_package("numpy==1.16.2")
dependencies.add_pip_package("pandas==0.23.4")
dependencies.add_pip_package("joblib==0.14.1")
compute_config.environment.python.conda_dependencies = dependencies


### Running the experiment

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=".",script="train.py", run_config=compute_config)

experiment_name = "reg-exp"
experiment = Experiment(workspace=ws, name=experiment_name)

run = experiment.submit(config=script_run_config)

run.wait_for_completion(show_output=True)

In [ ]:
run.get_file_names()

In [ ]:
run.download_file(name=model_dir + "/" + model_name)

### Registering model

In [ ]:
from azureml.core.model import Model

Model.register(workspace=ws, model_path=model_name, model_name="regression_model")
